This notebook is heavily inspired by [link](https://www.youtube.com/watch?v=aywZrzNaKjs&ab_channel=Rabbitmetrics)


In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

#### LLM WRAPPER

In [2]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")
llm("explain large language models in one sentence")

'\n\nLarge language models are neural networks trained on large datasets of natural language text, used to generate predictions about the probability of a given word or phrase appearing in a sentence.'

In [3]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [4]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a python script that does sagemaker batch processing inside lambda function")
]
response=chat(messages)

In [ ]:
print(response.content, end='\n')

Here is a sample Python script that performs SageMaker batch processing inside a Lambda function:

```python
import boto3
import os

def lambda_handler(event, context):
    # Set up the SageMaker client
    sm_client = boto3.client('sagemaker')

    # Set up the input and output S3 locations
    input_s3_uri = 's3://<input-bucket>/<input-prefix>'
    output_s3_uri = 's3://<output-bucket>/<output-prefix>'

    # Set up the batch job parameters
    job_name = '<batch-job-name>'
    job_input = {
        'S3InputMode': 'File',
        'S3Uri': input_s3_uri,
        'InputMode': 'File',
        'ContentType': 'text/csv'
    }
    job_output = {
        'S3OutputPath': output_s3_uri,
        'OutputName': 'output'
    }
    job_args = {
        'ModelName': '<model-name>',
        'MaxPayloadInMB': 6
    }
    job_config = {
        'BatchStrategy': 'MultiRecord',
        'MaxConcurrentTransforms': 0,
        'MaxPayloadInMB': 6
    }

    # Start the batch job
    response = sm_client.crea

#### PROMT TEMPLATES

In [ ]:
from langchain import PromptTemplate

template = """
You are an expert data scientist with an expetise in building ML model in AWS ecosystem.
Explain the concept of {concept} in a couple of lines
"""

promt = PromptTemplate(
    input_variables=["concept"],
    template=template
)

In [ ]:
promt

PromptTemplate(input_variables=['concept'], output_parser=None, partial_variables={}, template='\nYou are an expert data scientist with an expetise in building ML model in AWS ecosystem.\nExplain the concept of {concept} in a couple of lines\n', template_format='f-string', validate_template=True)

In [ ]:
llm(promt.format(concept="batch transform"))

'\nBatch Transform is an AWS SageMaker feature that enables the user to quickly and cost-effectively process large amounts of data in a single request. It allows for batch inference on an entire dataset using an existing ML model. This means that it can quickly generate inferences on a large dataset without the need to manually pre-process or set up a separate training job.'

In [ ]:
# HERE we dynamically change the promt with user input
llm(promt.format(concept="model registry"))

'\nModel registry is a centralized platform for managing and tracking the different versions of ML models. It provides a single source of truth for recording and tracking model versions, data sets, and model-related artifacts. This helps organizations to manage model lineage, track model performance, and enable reproducibility.'

#### CHAIN
It takes LLM and Template and combines them into an interface that takes input from the user and returnds output from LLM

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=promt)

# Run the chain only specidying the input vatiable
print(chain.run("autoencoder"))


Autoencoders are a type of neural network that is used for unsupervised learning. It is an unsupervised algorithm that takes an input, encodes it into a hidden representation, and then decodes the hidden representation back to its original form. Autoencoders are used for a variety of tasks such as dimensionality reduction, anomaly detection, and image reconstruction.


In [ ]:
# the second chain takes the output of the first chain as input and does an action on it (in this case it 
# takes a concept from first chain and explains it to me like I'm fine years old)

second_promt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Trun the concept description of {ml_concept} and explain it to me like I am five in 100 words"
)
chain_two = LLMChain(llm=llm, prompt=second_promt)

In [ ]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain
explanation = overall_chain.run("autoencoder")
print(explanation)



> Entering new SimpleSequentialChain chain...

An autoencoder is a type of artificial neural network used to learn efficient representations of input data, called "codings", by reconstructing the input data from the codings. It is designed to reduce the dimensionality of the input data while preserving the important features of the data in the codings.


An autoencoder is a type of machine learning. It helps computers to figure out patterns in data. It works by taking in a set of data and compressing it into a smaller set of numbers, called codings. This helps computers to understand the data better, without losing any important information. The codings can then be used to reconstruct the data, which is how the autoencoder learns. It's a great way of reducing a lot of data into something much more manageable.

> Finished chain.


An autoencoder is a type of machine learning. It helps computers to figure out patterns in data. It works by taking in a set of data and compressing it into

#### Embedding and VectorStores
Split the explanation into chunks to be stored in VectorStore - "pincone"

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 50,
    chunk_overlap = 0,
)
text = text_splitter.create_documents([explanation])

In [ ]:
text

[Document(page_content='An autoencoder is a type of machine learning. It', metadata={}),
 Document(page_content='helps computers to figure out patterns in data. It', metadata={}),
 Document(page_content='works by taking in a set of data and compressing', metadata={}),
 Document(page_content='it into a smaller set of numbers, called codings.', metadata={}),
 Document(page_content='This helps computers to understand the data', metadata={}),
 Document(page_content='better, without losing any important information.', metadata={}),
 Document(page_content='The codings can then be used to reconstruct the', metadata={}),
 Document(page_content="data, which is how the autoencoder learns. It's a", metadata={}),
 Document(page_content='great way of reducing a lot of data into something', metadata={}),
 Document(page_content='much more manageable.', metadata={})]

In [ ]:
text[0].page_content

'An autoencoder is a type of machine learning. It'

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model_name="ada")

In [ ]:
# vector representation of 0s embedding
query_results = embeddings.embed_query(text[0].page_content)
print(query_results[:5])
print(len(query_results))

[-0.03356080542448993, 0.02426007791594276, 0.015185698117317289, 0.016173385277295175, 0.0388078954991406]
1024


In [ ]:
import os
import pinecone
from langchain.vectorstores import Pinecone

pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENV')
)

In [ ]:
print(pinecone)

<module 'pinecone' from '/opt/anaconda3/envs/langchain-sandbox/lib/python3.11/site-packages/pinecone/__init__.py'>


In [ ]:
# we need to create an index first; use the lenght of 0s embedding as a dimension
dimension = len(query_results)
index_name = "langchain-quickstart"
pinecone.create_index(index_name, dimension=dimension)

In [ ]:
search = Pinecone.from_documents(text, embeddings, index_name=index_name)

In [ ]:
query = "What is magical about an autoencoder?"
result = search.similarity_search(query)

In [ ]:
# All the relevant chunks are extracted
result

[Document(page_content='An autoencoder is a type of machine learning. It', metadata={}),
 Document(page_content="data, which is how the autoencoder learns. It's a", metadata={}),
 Document(page_content='This helps computers to understand the data', metadata={}),
 Document(page_content='The codings can then be used to reconstruct the', metadata={})]

#### AGENT

In [ ]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

In [ ]:
agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=500),
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
# This allows LLM to run python code
agent_executor.run("Find the roots (zeros) if the quadratic function 3 * x**2 + 2*x - 1")



> Entering new AgentExecutor chain...
 I need to solve a quadratic equation
Action: Python REPL
Action Input: import numpy as np
Observation: 
Thought: I can use the numpy function to solve the equation
Action: Python REPL
Action Input: np.roots([3, 2, -1])
Observation: 
Thought: I now know the final answer
Final Answer: (-1.0, 0.3333333333333333)

> Finished chain.


'(-1.0, 0.3333333333333333)'

In [ ]:
# remove the index since free version of Pinecode allows only one index to be stored
index_name = "langchain-quickstart"
pinecone.delete_index(index_name)